In [24]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

palette = sns.color_palette(["#2176AB", "#F97662", "#FFBF00", "#50C878", "#B284BE"])
sns.set_palette(palette)

df = pd.read_csv('../../data/archive/ST001937_AN003150.csv')

Data preprocessing and cleaning

In [25]:
df.drop(columns=['Sample ID','RAW_FILE_NAME'], inplace=True)

#df healthy and cancer where 0 is healthy and 1 is cancer malignant or benign
df_healthy_cancer = df.copy()
df_healthy_cancer['Phenotypes'] = df_healthy_cancer['Phenotypes'].map({'Healthy Controls': 0, 'Benign SPNS': 1, 'Maligant SPNS': 1})
df_healthy_cancer.value_counts('Phenotypes')

Phenotypes
1    880
0    280
dtype: int64

In [26]:

#df benign and malignant where 0 is benign and 1 is malignant
df_benign_malignant = df.drop(df[df['Phenotypes'] == 'Healthy Controls'].index)
df_benign_malignant['Phenotypes'] = df_benign_malignant['Phenotypes'].map({'Benign SPNS': 0, 'Maligant SPNS': 1})
df_benign_malignant.value_counts('Phenotypes')

Phenotypes
1    723
0    157
dtype: int64

Standardizing the data

In [27]:
from sklearn import preprocessing

#standardizing the data excluding the Diagnosis column
standard_scaler = preprocessing.StandardScaler()

hca_phenotypes = df_healthy_cancer['Phenotypes']
bm_phenotypes = df_benign_malignant['Phenotypes']

df_healthy_cancer_no_diagnosis = df_healthy_cancer.drop('Phenotypes', axis=1)
df_benign_malignant_no_diagnosis = df_benign_malignant.drop('Phenotypes', axis=1)

hca_df = pd.DataFrame(standard_scaler.fit_transform(df_healthy_cancer_no_diagnosis), columns=df_healthy_cancer_no_diagnosis.columns)
bm_df = pd.DataFrame(standard_scaler.fit_transform(df_benign_malignant_no_diagnosis), columns=df_benign_malignant_no_diagnosis.columns)


hca_df['Phenotypes'] = hca_phenotypes
bm_df['Phenotypes'] = bm_phenotypes

hca_df.describe()

,"1,3,5(10)-estratrien-3,6- beta-17-beta-triol","1,5-anhydroglucitol",17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-1,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-2,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-3,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-4,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-5,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-6,1-hexadecanol,1-hydroxyanthraquinone prod,...,tyrosine-2,urea-1,urea-2,urea-3,urea-4,uridine,valine,xanthine,xanthosine,Phenotypes
count,1.160000e+03,1160.000000,1.160000e+03,1.160000e+03,1.160000e+03,1.160000e+03,1.160000e+03,1.160000e+03,1.160000e+03,1.160000e+03,...,1.160000e+03,1.160000e+03,1.160000e+03,1.160000e+03,1.160000e+03,1.160000e+03,1.160000e+03,1.160000e+03,1.160000e+03,1160.000000
mean,-1.225074e-17,0.000000,-1.607909e-16,-4.900295e-17,-7.503576e-17,-9.188053e-18,-1.225074e-17,6.125368e-17,2.388894e-16,2.450147e-17,...,1.225074e-17,1.684476e-16,-1.500715e-16,1.531342e-16,1.531342e-18,-1.225074e-17,1.225074e-17,1.225074e-17,-4.900295e-17,0.758621
std,1.000431e+00,1.000431,1.000431e+00,1.000431e+00,1.000431e+00,1.000431e+00,1.000431e+00,1.000431e+00,1.000431e+00,1.000431e+00,...,1.000431e+00,1.000431e+00,1.000431e+00,1.000431e+00,1.000431e+00,1.000431e+00,1.000431e+00,1.000431e+00,1.000431e+00,0.428104
min,-1.942518e+01,-9.940976,-1.236017e+00,-1.029234e+01,-8.241675e-01,-1.877206e+01,-1.069124e+00,-1.053430e+00,-1.895002e+00,-1.189001e+00,...,-1.581639e+00,-1.614048e+00,-1.355858e+00,-1.765960e+00,-3.189689e-01,-1.470885e+00,-6.725485e+00,-3.204565e-01,-2.099787e+01,0.000000
25%,-5.704542e-02,-0.187383,-5.046493e-01,-3.383285e-01,-7.294057e-01,-2.809717e-02,-5.271820e-01,-5.141986e-01,-3.879349e-01,-5.199062e-01,...,-4.687686e-01,-5.177106e-01,-6.275440e-01,-5.467868e-01,-2.960466e-01,-1.133873e-01,-2.420243e-01,-2.384552e-01,-2.995882e-01,1.000000
50%,-4.631734e-02,-0.031350,-1.984306e-01,-2.399466e-01,-1.796837e-01,7.292530e-03,-1.703977e-01,-1.943191e-01,-8.700994e-02,-2.484115e-01,...,-9.492164e-02,-1.062400e-01,-1.183819e-01,-1.176760e-01,-2.768648e-01,-1.075314e-01,-1.902063e-01,-1.568511e-01,-1.567816e-01,1.000000
75%,-6.019838e-03,0.134891,2.421417e-01,2.934771e-02,4.299368e-01,4.657662e-02,2.657046e-01,2.554157e-01,1.912782e-01,2.528499e-01,...,3.678364e-01,3.786152e-01,4.831423e-01,3.583828e-01,-2.222636e-01,-8.608978e-02,-5.532306e-02,-1.482252e-02,1.799523e-01,1.000000
max,1.310926e+01,24.667658,1.339818e+01,1.176364e+01,1.230419e+01,2.076315e+01,2.056656e+01,1.494297e+01,1.407417e+01,8.887802e+00,...,1.721807e+01,1.500379e+01,1.656874e+01,1.646400e+01,1.389422e+01,2.604879e+01,1.738537e+01,2.441393e+01,1.188097e+01,1.000000


Class balancing

In [28]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
import pandas as pd

# Separate input features and target
y = hca_df['Phenotypes']
X = hca_df.drop('Phenotypes', axis=1)

# 1 = 880, 0 = 280 ->  1 = 500, 0 = 500

# Setting up SMOTE for oversampling
smote = SMOTE(sampling_strategy={0: 500}, random_state=42)
X_res_over, y_res_over = smote.fit_resample(X, y)

# Setting up ClusterCentroids for undersampling
cc = ClusterCentroids(sampling_strategy={1: 500}, random_state=42)
X_res_under, y_res_under = cc.fit_resample(X_res_over, y_res_over)

# Concatenate our training data back together
res_df = pd.concat([X_res_under, y_res_under], axis=1)

# Check the new class counts
res_df['Phenotypes'].value_counts()
res_df.to_csv('../../data/ST001937_hca.csv', index=False)

/home/marco/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [29]:
y = bm_df['Phenotypes']
X = bm_df.drop('Phenotypes', axis=1)

# 1 = 723, 0 = 157 ->  1 = 300, 0 = 300

# Setting up SMOTE for oversampling
smote = SMOTE(sampling_strategy={0: 300}, random_state=42)
X_res_over, y_res_over = smote.fit_resample(X, y)

# Setting up ClusterCentroids for undersampling
cc = ClusterCentroids(sampling_strategy={1: 300}, random_state=42)
X_res_under, y_res_under = cc.fit_resample(X_res_over, y_res_over)

# Concatenate our training data back together
res_df = pd.concat([X_res_under, y_res_under], axis=1)

# Check the new class counts
res_df['Phenotypes'].value_counts()
res_df.to_csv('../../data/ST001937_bm.csv', index=False)

/home/marco/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
